In [1]:
import pandas as pd
import numpy as np
from six.moves import urllib
import tensorflow as tf
import word2vec
import re
import tempfile

In [3]:
# train word2vec with sample word list dataset
# this might take a minute
vecfile = '/home/claire/Documents/jobs/job-hunting/massively/text8'
word2vec.word2vec(vecfile, '/home/claire/Documents/jobs/job-hunting/massively/text8.bin', size=100, verbose=True)
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

Starting training using file /home/claire/Documents/jobs/job-hunting/massively/text8
Vocab size: 71291
Words in train file: 16718843
Alpha: 0.000002  Progress: 100.03%  Words/thread/sec: 188.12k  

In [4]:
# load the vector model
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

In [94]:
# import the sentiment data
df = pd.DataFrame.from_csv('/home/claire/Documents/jobs/job-hunting/massively/sentiment_analysis/dfe_happysad_utf.csv', header=0, sep=',', index_col=0)

In [58]:
# split the data into test and train
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

df_train = df[msk]
df_test = df[~msk]

In [59]:
# split training and testing tweets by sadnesss and happiness
sad_train = df_train.loc[df_train['label'] == 'sadness']
happy_train = df_train.loc[df_train['label'] == 'happiness']

sad_test = df_test.loc[df_test['label'] == 'sadness']
happy_test = df_test.loc[df_test['label'] == 'happiness']

In [60]:
# create numpy arrays
np_sad_train = sad_train.as_matrix(columns=df.columns[0:2])
np_happy_train = happy_train.as_matrix(columns=df.columns[0:2])

np_sad_test = sad_test.as_matrix(columns=df.columns[0:2])
np_happy_test = happy_test.as_matrix(columns=df.columns[0:2])

# or create objects for mean embedding vectorizer

In [62]:
# set maximum sentence length and number of dimension for each word vector
maxSeqLength = df_train['features'].str.len().max()
numDimensions = 300 

In [63]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def newline(string):
    newline_pattern = re.compile("[^\n]*\n")
    return re.findall(newline_pattern, string)

In [123]:
firstFile = np.zeros((maxSeqLength), dtype='int32')

for tweet in np_sad_train:
    indexCounter = 0
    cleaned = cleanSentences(tweet[0])
    split = cleaned.split()
    print(split)
    for word in split:
        try:
            firstFile[indexCounter] = model[word]
        except KeyError:
            firstFile[indexCounter] = 399999 #Vector for unknown words
    indexCounter = indexCounter + 1
firstFile

['layin', 'n', 'bed', 'with', 'a', 'headache', 'ughhhhwaitin', 'on', 'your', 'call']


ValueError: setting an array element with a sequence.

In [168]:
i = np.where( model.vocab == 'the')
i[0]


array([1])

In [161]:
# use scikit-learn to pipeline actual models that will take tokenised text, 
# vectorize and learn to classify the vectors

In [ ]:
# classify the vectors with a classification model -- SVM, Naive Multinomial Bayesian
# can also do the same thing with weights -- TF-IDF